<h3>Парсинг объявлений о продаже автомобилей с Авито</h3>

In [29]:
# импорт библиотек
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
from bs4 import BeautifulSoup

In [30]:
# начальная страница парсинга
url = 'https://www.avito.ru/samarskaya_oblast/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ'

Напишем функцию для решения Captcha

In [31]:
def kill_captcha(driver):
    wait = WebDriverWait(driver, 5)
    try:
        # Проверка на наличие капчи
        captcha_element = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//h2[@class='firewall-title']")), message="Капча обнаружена")

        if captcha_element:
            print("Капча обнаружена. Решите ее вручную.")
            input("Нажмите Enter, когда решите капчу...")

    except TimeoutException:
        pass

Напишем функцию, которая будет закрывать страницу, если объявление недоступно

In [32]:
def is_unavailable_ad(driver):
    wait = WebDriverWait(driver, 5)
    try:
        # Проверка на недоступное объявление
        blocked_ad = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//span[text()='Перейти к поиску']")))

        if blocked_ad:
            print("Недоступное объявление!")
            driver.find_element(
                By.XPATH, "//span[text()='Перейти к поиску']").click()
            return True

    except TimeoutException:
        return False

Напишем функцию получения данных со страницы объявления 

In [33]:
def get_car_data(driver):
    wait = WebDriverWait(driver, 5)  

    # Проверка на капчу
    kill_captcha(driver)

    # Проверка на недоступное объявление 
    if is_unavailable_ad(driver):
        return

    # Ожидание загрузки элементов с информацией о машине
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//span[@itemprop='name']"))) 
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//li[@class='params-paramsList__item-_2Y2O']"))) 
    wait.until(EC.presence_of_element_located((By.XPATH, "//span[@itemprop='price']")))

    car_data = {}

    # Парсинг объявления
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    base_info = [x.text.strip() for x in soup.find_all('span', itemprop='name')]
    mark, model = base_info[-3], base_info[-2]

    specs = [x.text.strip() for x in soup.find_all('li', class_='params-paramsList__item-_2Y2O')]
    car_data = {item.split(': ')[0]: item.split(': ')[1] for item in specs}
    car_data['Название'] = mark + " " + model
    car_data['Марка'] = mark

    # Получение цены с использованием явного ожидания
    price_element = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@itemprop='price']")))
    car_data['Цена'] = price_element.get_attribute('content')

    return car_data

Напишем фукнцию парсинга всех объявлений

In [34]:
def parse_cars(url, count_pages=1):
    driver = webdriver.Chrome()

    driver.get(url)

    # Ожидание загрузки страницы
    wait = WebDriverWait(driver, 10)

    # Проверка на капчу
    kill_captcha(driver)

    wait.until(EC.presence_of_element_located(
        (By.XPATH, "//div[span[@class='styles-module-switcherCircle-NWY_O']]")))

    # Включение фильтра "Сначала из Самарской области"
    driver.find_element(
        By.XPATH, "//div[span[@class='styles-module-switcherCircle-NWY_O']]").click()

    # Ожидание загрузки кнопки "Показать ... объявлений"
    wait.until(EC.presence_of_element_located(
        (By.XPATH, "//button[@data-marker='search-filters/submit-button']")))

    # Клик по кнопке "Показать ... объявлений"
    driver.find_element(
        By.XPATH, "//button[@data-marker='search-filters/submit-button']").click()

    cars = []

    # Парсинг всех страниц с объявлениями
    for cur_page in range(count_pages):
        # Ожидание загрузки ссылок на объявления
        wait.until(EC.presence_of_all_elements_located(
            (By.XPATH, "//a[@itemprop='url' and @data-marker='item-title']")))

        # Сбор ссылок на объявления
        car_links = [x.get_attribute("href") for x in driver.find_elements(
            By.XPATH, "//a[@itemprop='url' and @data-marker='item-title']")]

        # Парсинг каждого объявления на странице
        for car_link in car_links:
            # Открытие страницы объявления
            driver.execute_script(f"window.open('{car_link}');")

            # Переключение на новое окно
            driver.switch_to.window(driver.window_handles[-1])

            # Проверка на капчу
            kill_captcha(driver)

            # Получение данных со страницы объявления
            car_data = get_car_data(driver)

            # Добавление данных в список объявлений
            if car_data:
                cars.append(car_data)

            # Закрытие окна объявления
            driver.close()

            # Переключение на исходное окно
            driver.switch_to.window(driver.window_handles[0])

        if cur_page != count_pages - 1:
            # Ожидание загрузки кнопки "Следующая страница"
            wait.until(EC.presence_of_element_located(
                (By.XPATH, "//a[@aria-label='Следующая страница']")))
            driver.find_element(
                By.XPATH, "//a[@aria-label='Следующая страница']").click()

    driver.quit()

    return cars

In [35]:
# Парсинг
cars = parse_cars(url, 100)

Недоступное объявление!
Недоступное объявление!
Недоступное объявление!
Недоступное объявление!
Недоступное объявление!


Преобразуем собранные данные в **DataFrame** и сохраним в файл

In [38]:
import pandas as pd

cars = pd.DataFrame(cars)

cars.to_csv('../data/cars_data.csv')

In [ ]:
cars.head()

,Год выпуска,Поколение,Пробег,История пробега,ПТС,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Название,Цена,Обмен
0,2010,I рестайлинг (2009—2012),156 000 км,7 записей в отчёте Автотеки,Оригинал,2,Не битый,2.3 4WD AT (238 л.с.),2.3 л,Бензин,Автомат,Полный,Sport,Внедорожник 5-дверный,Белый,Левый,JMZE*************,Mazda CX-7,1050000,NaN
1,2020,G07 (2018—2022),109 000 км,Проверить в Автотеке,Оригинал,1,Не битый,M50d 3.0 xDrive Steptronic (400 л.с.),3 л,Дизель,Автомат,Полный,Special by Individual,Внедорожник 5-дверный,Синий,Левый,WBAC*************,BMW X7,8800000,Не интересует
2,2023,I (2023—2024),15 км,NaN,NaN,1,Не битый,2.0 T-GDI 4WD DCT (249 л.с.),2 л,Бензин,Робот,Полный,Platinum,Внедорожник 5-дверный,Чёрный,Левый,LVTD*************,EXEED RX,3730000,NaN
3,2024,I (2022—2024),300 км,2 записи в отчёте Автотеки,NaN,1,Не битый,1.5hyb 4WD AT (530 л.с.),1.5 л,Гибрид,Автомат,Полный,Executive,Седан,Чёрный,Левый,LDP9*************,Voyah Passion (Chasing Light),6770000,NaN
4,2024,I (2021—2024),10 км,NaN,Электронный,1,Не битый,2.0 4WD AT (238 л.с.),2 л,Бензин,Автомат,Полный,Flagship,Внедорожник 5-дверный,Белый,Левый,LB37*************,Geely Monjaro,3600000,Не интересует
